In [16]:
import os
import json
import sqlite3
import gradio as gr
from dotenv import load_dotenv
from google import genai
from google.genai import types

In [17]:
load_dotenv(override=True)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
MODEL = "gemini-2.5-flash"

client = genai.Client(api_key=GEMINI_API_KEY)

In [18]:
system_message = """
You are an AI assistant for an airline company called Flightly. 
Give short, courteous answers, no more than one sentence. 
Always be polite and professional in your responses.
You only know about airline data. If you don't know the answer, say so.
Assume that all flyghts are from a unique origin, so each flight has a unique price associated with the city"""

In [19]:
def chat(message, history):
    google_messages = []

    google_messages.append({"role": "user", "parts": [{"text": system_message}]})
    google_messages.append({"role": "model", "parts": [{"text": "Entendido, seguiré esas instrucciones."}]})

    for h in history:
        role = "user" if h["role"] == "user" else "model"
        google_messages.append({
            "role": role,
            "parts": [{"text": h["content"]}]
        })
    google_messages.append({"role": "user", "parts": [{"text": message}]})
    response = client.models.generate_content(
        model=MODEL,
        contents=google_messages
    )
    
    return response.text

gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [20]:
ticket_prices = {"london": "$800", "paris": "$900", "tokyo": "$1400", "berlin": "$500"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknow ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [21]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $800'

In [22]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [23]:
tools = [types.Tool(function_declarations=[
    types.FunctionDeclaration(
        name="get_ticket_price",
        description="Get the price of a return ticket to the destination city",
        parameters=types.Schema(
            type=types.Type.OBJECT,
            properties={"destination_city": types.Schema(type=types.Type.STRING, description="The city that the customer wants to travel to")},
            required=["destination_city"]
        )
    ),
    types.FunctionDeclaration(
        name="set_ticket_price",
        description="Set the price of a return ticket to the destination city",
        parameters=types.Schema(
            type=types.Type.OBJECT,
            properties={
                "destination_city": types.Schema(type=types.Type.STRING, description="The city that the customer wants to travel to"),
                "price": types.Schema(type=types.Type.NUMBER, description="The price of the ticket")
            },
            required=["destination_city", "price"]
        )
    )
])]

In [24]:
tools

[Tool(
   function_declarations=[
     FunctionDeclaration(
       description='Get the price of a return ticket to the destination city',
       name='get_ticket_price',
       parameters=Schema(
         properties={
           'destination_city': Schema(
             description='The city that the customer wants to travel to',
             type=<Type.STRING: 'STRING'>
           )
         },
         required=[
           'destination_city',
         ],
         type=<Type.OBJECT: 'OBJECT'>
       )
     ),
     FunctionDeclaration(
       description='Set the price of a return ticket to the destination city',
       name='set_ticket_price',
       parameters=Schema(
         properties={
           'destination_city': Schema(
             description='The city that the customer wants to travel to',
             type=<Type.STRING: 'STRING'>
           ),
           'price': Schema(
             description='The price of the ticket',
             type=<Type.NUMBER: 'NUMBER'>
       

In [25]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [27]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    elif tool_call.function.name == "set_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = arguments.get('price')
        set_ticket_price(city, price)
        response = {
            "role": "tool",
            "content": f"Price for {city} set to ${price}",
            "tool_call_id": tool_call.id
        }
    return response

In [28]:
def chat(message, history):
    gemini_history = []
    if not history:
        gemini_history.append({"role": "user", "parts": [{"text": system_message}]})
    for h in history:
        role = "user" if h["role"] == "user" else "model"
        if isinstance(h["content"], list):
            parts = [{"text": item["text"]} for item in h["content"] if item.get("type") == "text"]
        else:
            parts = [{"text": h["content"]}]
        gemini_history.append({"role": role, "parts": parts})
    config = types.GenerateContentConfig(tools=tools)
    chat_session = client.chats.create(model=MODEL, config=config, history=gemini_history)
    response = chat_session.send_message(message)
    part = response.candidates[0].content.parts[0]
    if hasattr(part, 'function_call') and part.function_call:
        call = part.function_call
        if call.name == 'get_ticket_price':
            city = call.args['destination_city']
            price = get_ticket_price(city)
            tool_part = types.Part(function_response=types.FunctionResponse(name=call.name, response={"result": price}))
            tool_response = chat_session.send_message(tool_part)
            return tool_response.candidates[0].content.parts[0].text
        elif call.name == 'set_ticket_price':
            city = call.args['destination_city']
            price = call.args['price']
            set_ticket_price(city, price)
            tool_part = types.Part(function_response=types.FunctionResponse(name=call.name, response={"result": f"Price for {city} set to ${price}"}))
            tool_response = chat_session.send_message(tool_part)
            return tool_response.candidates[0].content.parts[0].text
    return part.text

In [29]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [30]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [31]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [32]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'Ticket price to London is $$800'

In [33]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [34]:
ticket_prices = {"london": "$800", "paris": "$900", "tokyo": "$1400", "berlin": "$500"}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [35]:
get_ticket_price("Tokyo")

DATABASE TOOL CALLED: Getting price for Tokyo


'Ticket price to Tokyo is $$1400'

In [36]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
